In [1]:
import torch
from modelscope import AutoTokenizer, AutoModel
from PIL import Image
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)


def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform


def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio


def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images


def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values


def preprocess_image(file_path, dynamic=True, max_num=6, image_size=448):
    try:
        if dynamic:
            return load_image(file_path, max_num=max_num).to(torch.bfloat16).cuda()
        else:
            img = Image.open(file_path).convert('RGB')
            transform = build_transform(image_size)
            pixel_values = transform(img)
            return torch.stack([pixel_values]).to(torch.bfloat16).cuda()
    except Exception as e:
        raise RuntimeError(f"Error processing image: {e}")


path = "Reyes-8B"

model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
).eval().cuda()

# print(model)

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)
generation_config = dict(max_new_tokens=2048, do_sample=False)


/home/jovyan/yjhllm/miniconda3/envs/ccks/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# 方式1：串行推理

In [20]:

file_path = 't6.png'
pixel_values = preprocess_image(file_path, dynamic=True)
question = '<image>\nDescribe this image.'
response = model.chat(tokenizer, pixel_values, question, generation_config)
print(f'User: {question}\nAssistant: {response}')


file_path = 't6.png'
pixel_values = preprocess_image(file_path, dynamic=True)
question = '<image>\nDescribe this image.'
response = model.chat(tokenizer, pixel_values, question, generation_config)
print(f'User: {question}\nAssistant: {response}')


file_path = 't6.png'
pixel_values = preprocess_image(file_path, dynamic=True)
question = '<image>\nDescribe this image.'
response = model.chat(tokenizer, pixel_values, question, generation_config)
print(f'User: {question}\nAssistant: {response}')


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


User: <image>
Describe this image.
Assistant: The image captures a lively scene on a basketball court, with two men in yellow and purple jerseys standing in the center of the action. One of the players is wearing a yellow shirt with the number 23 on it, while the other is wearing a yellow shirt with the number 28 on it. They are both standing in front of a crowd of fans, who are watching the game with excitement.

There are several other people in the background, some of whom might be teammates, while others could be coaches or fans. The atmosphere is energetic and full of anticipation as the game unfolds.


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


User: <image>
Describe this image.
Assistant: The image captures a lively scene on a basketball court, with two men in yellow and purple jerseys standing in the center of the action. One of the players is wearing a yellow shirt with the number 23 on it, while the other is wearing a yellow shirt with the number 28 on it. They are both standing in front of a crowd of fans, who are watching the game with excitement.

There are several other people in the background, some of whom might be teammates, while others could be coaches or fans. The atmosphere is energetic and full of anticipation as the game unfolds.
User: <image>
Describe this image.
Assistant: The image captures a lively scene on a basketball court, with two men in yellow and purple jerseys standing in the center of the action. One of the players is wearing a yellow shirt with the number 23 on it, while the other is wearing a yellow shirt with the number 28 on it. They are both standing in front of a crowd of fans, who are watc

# 方式2：batch推理

In [24]:
import nvtx
import time 


questions = [
    "<image>\nDescribe this image.",
    "<image>\nDescribe this image.",
    "<image>\nDescribe this image.",
]

images_path = ["t6.png","t6.png","t6.png"]


def conversation(model, tokenizer, questions, images_path,generation_config,histories):
    pixel_values_list=[]

    for i in range(len(questions)):
        if images_path[i] is not None:
            pixel_values = preprocess_image(file_path, dynamic=True)
            pixel_values_list.append(pixel_values)


    return model.chat_batch(tokenizer, pixel_values_list, questions, generation_config, histories, return_histories=False)

responses= conversation(model, tokenizer, questions, images_path,generation_config,histories=None)
for question, response in zip(questions, responses):
    print(f"User: {question}\nAssistant: {response}\n")


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


User: <image>
Describe this image.
Assistant: The image captures a lively scene on a basketball court, with two men in yellow and purple jerseys standing in the center of the action. One of the players is wearing a yellow shirt with the number 23 on it, while the other is in a yellow and purple jersey. They are both standing near each other, celebrating a successful play or cheering on their teammates.

In the background, there are several other people, likely teammates or opponents, scattered across the court. Some of them are closer to the foreground, while others are further away in the background. The overall atmosphere of the scene is energetic and full of excitement, typical of a thrilling basketball game.

User: <image>
Describe this image.
Assistant: The image captures a lively scene on a basketball court, with two men in yellow and purple jerseys standing in the center of the action. One of the players is wearing a yellow shirt with the number 23 on it, while the other is in a